In [9]:
import geopandas as gpd
import pandas as pd
import numpy as np
from dotenv import find_dotenv, load_dotenv
import os
from arcgis import GIS

In [10]:
load_dotenv(find_dotenv())

gis_user = os.getenv('ESRI_USERNAME')
gis_pass = os.getenv('ESRI_PASSWORD')
gis_url = os.getenv('ESRI_URL')

gisE = GIS(url=gis_url, username=gis_user, password=gis_pass)
gisE

GIS @ https://co.kent.de.us/portal/home version:5.1